## Imports 

In [1]:
import numpy as np
import os
import random
from random import choice 
from copy import deepcopy

## Reading databases

In [2]:
def read_databases(filename):
    x1 = []
    x2 = []
    y = []
    for line in open(filename):
        csv_row = line.strip().split(',')
        csv_row = [float(number) for number in csv_row]
        x1.append(csv_row[0])
        x2.append(csv_row[1])
        y.append(csv_row[2])
    return x1, x2, y

In [3]:
filename = 'datasets/synth1/synth1-test.csv'
x1_train, x2_train, y_train = read_databases(filename)

In [4]:
filename = 'datasets/synth1/synth1-train.csv'
x1_test, x2_test, y_test = read_databases(filename)

## Defining individual class

In [5]:
expression = [['expression', 'op', 'expression'], 'variable', 'value']
operation = ['+', '-', '*']
variable = ['x1', 'x2']
value = ['random']

In [6]:
terminals_dict = dict()
terminals_dict['op'] = operation
terminals_dict['variable'] = variable
terminals_dict['value'] = value

In [7]:
class Node(object):
    def __init__(self, data='op', level=0):
        self.data = data
        self.type = data
        self.left_child = None
        self.right_child = None
        self.level = level
        self.max_level = 3
        self.terminal = False
    
    def __copy__(self):
        clone = Node()
        clone.data = self.data
        clone.type = self.type
        clone.left_child = self.left_child
        clone.right_child = self.right_child
        clone.level = self.level
        clone.max_level = self.max_level
        clone.terminal = seld.terminal

    def initial_node(self):
        self.add_left('expression')
        self.add_right('expression')
    
    def initialize(self):
        self.add_left('expression')
        self.add_right('expression')
        self.expand_children()
        self.get_parameters()

    def add_left(self, value):
        self.left_child = Node(value, level=self.level+1)
    
    def add_right(self, value):
        self.right_child = Node(value, level=self.level+1)
        
    def show(self):
        print(self.level, ":  ", self.data)
        if self.left_child:
            self.left_child.show()
            self.right_child.show()
    
    def get_expression(self):
        if self.data == 'expression':
            self.data = choice(expression)
            if type(self.data) == list:
                if self.level < self.max_level: 
                    self.add_left(self.data[0])
                    self.add_right(self.data[2])
                    self.data = self.data[1]
                else:
                    self.data = 'value'
                    
        if self.data != 'op':
            self.terminal = True
                    
        if self.left_child is not None:
            self.left_child.get_expression()
        
        if self.right_child is not None:
            self.right_child.get_expression()
        
        self.type = self.data
    
    def expand_children(self):
        self.left_child.get_expression()
        self.right_child.get_expression()
    
    def get_parameters(self):
        self.data = choice(terminals_dict[self.data])
        if self.data == 'random':
            self.data = random.random()*5
        
        if self.left_child is not None:
            self.left_child.get_parameters()
        
        if self.right_child is not None:
            self.right_child.get_parameters()
    
    def evaluate(self, x1=0, x2=0):
        
        if self.terminal:
            if self.data == 'x1':
                return x1
            elif self.data == 'x2':
                return x2
            else:
                return self.data
            
        if self.left_child is not None:
            result_left = self.left_child.evaluate(x1, x2)
        if self.right_child is not None:
            result_right = self.right_child.evaluate(x1, x2)
        
        if self.data == '+':
            return result_left + result_right
        elif self.data == '-':
            return result_left - result_right
        elif self.data == '*':
            return result_left * result_right
    
    def error(self, x1, x2, y):
        return (y - self.evaluate(x1, x2))**2
    
    def fitness(self, x1_list, x2_list, y_list):
        error_sum = 0
        for x1, x2, y in zip(x1_list, x2_list, y_list):
            error_sum += self.error(x1, x2, y)
        y_mean = y_list - np.mean(y_list)
        y_mean_sum = sum(y_mean**2)
            
        return (error_sum/y_mean_sum)**0.5
    
    def change_node(self):
        self.data = choice(terminals_dict[self.type])
        if self.data == 'random':
            self.data = random.random()*5
    
    def mutate(self, m_level):
        if m_level == self.level:
            self.change_node()
        elif np.random.rand() < 0.5 and self.left_child is not None:
            self.left_child.mutate(m_level)
        elif self.right_child is not None:
            self.right_child.mutate(m_level)
        else:
            self.change_node()

In [8]:
root = Node()
root.initial_node()
root.expand_children()
root.show()

root.get_parameters()
print()
root.show()

root.evaluate()

0 :   op
1 :   op
2 :   value
2 :   op
3 :   value
3 :   variable
1 :   variable

0 :   +
1 :   -
2 :   3.8173556838867735
2 :   -
3 :   4.93542256421709
3 :   x1
1 :   x1


-1.1180668803303164

In [9]:
root.evaluate(2,5)

2.8819331196696836

In [10]:
root.error(2,5, 2)

0.7778060275703004

## Generating initial population

In [11]:
##Parameters
population_size = 5
crossover_probability = 0.9

In [12]:
class Genetic():
    def __init__(self, population_size, crossover_probability=0.9):
        if crossover_probability < 0 or crossover_probability > 1 :
            raise ValueError('Please give a valid number for crossover probability.')
        self.population_size = population_size
        self.crossover_probability = crossover_probability
        self.mutation_probability = 1 - crossover_probability
        self.population = []
        for i in range(self.population_size):
            root = Node()
            root.initialize()
            self.population.append(root)
    
    def sort_by_fitness(self, x1_list, x2_list, y_list):
        self.population.sort(key = lambda x: x.fitness(x1_list, x2_list, y_list))
    
    def rank_selection_pair(self):
        ## Assign probabilities inversely proportional to the position index
        p = list(range(len(solution1.population))) + np.array(1)
        p = np.flipud(p)  # reverse array
        p = p/sum(p)
        return np.random.choice(self.population,size=2, p=p) #Return two individuals from population
    
    def generate_offspring(self):
        if np.random.rand() < self.crossover_probability:
            ## Crossover
            return self.crossover()
        else:
            ## Mutation
            return self.mutation()
    
    def crossover(self):
        ## Chosing mutation level with a pobability 
        pass
    
    def mutation(self):
        parents = self.rank_selection_pair()
        children = [deepcopy(parents[0]), deepcopy(parents[1])]
        print(children[0].show())
        print(children[1].show())
        p = list(range(3)) + np.array(1)
        p = p/sum(p)
        for child in children:
            mutation_level = np.random.choice(3, p=p)
            child.mutate(mutation_level)
        return children

In [13]:
solution1 = Genetic(population_size, crossover_probability=0)

In [14]:
solution1.sort_by_fitness(x1_test, x2_test, y_test)

In [15]:
print(solution1.population[0].fitness(x1_test, x2_test, y_test))
solution1.population[0].show()

1.0847070889893258
0 :   *
1 :   -
2 :   2.3343129752704286
2 :   x1
1 :   2.577002726699573


In [16]:
print(solution1.population[1].fitness(x1_test, x2_test, y_test))
solution1.population[1].show()

1.1877029227629448
0 :   +
1 :   -
2 :   -
3 :   2.5165969153659837
3 :   1.1451155621575404
2 :   -
3 :   0.9636732602329606
3 :   0.1553201771090429
1 :   +
2 :   x1
2 :   x2


In [17]:
print(solution1.population[2].fitness(x1_test, x2_test, y_test))
solution1.population[2].show()

1.201960178822782
0 :   *
1 :   +
2 :   2.452524361514001
2 :   4.863017131336763
1 :   3.4557695466944183


In [18]:
print(solution1.population[3].fitness(x1_test, x2_test, y_test))
solution1.population[3].show()

1.2115674658317581
0 :   *
1 :   +
2 :   1.268341563203335
2 :   0.18825629132706634
1 :   x1


In [19]:
print(solution1.population[4].fitness(x1_test, x2_test, y_test))
solution1.population[4].show()

1.2134019899750847
0 :   -
1 :   x1
1 :   x1


## Selection

In [20]:
solution1.rank_selection_pair()

array([<__main__.Node object at 0x000002219B4F2E48>,
       <__main__.Node object at 0x000002219B4F2908>], dtype=object)

In [21]:
children = solution1.generate_offspring()

0 :   *
1 :   +
2 :   1.268341563203335
2 :   0.18825629132706634
1 :   x1
None
0 :   *
1 :   -
2 :   2.3343129752704286
2 :   x1
1 :   2.577002726699573
None


In [22]:
children[0].show()

0 :   *
1 :   +
2 :   1.268341563203335
2 :   0.18825629132706634
1 :   x2


In [23]:
children[1].show()

0 :   *
1 :   -
2 :   2.3343129752704286
2 :   x1
1 :   4.666599491635413
